<a href="https://colab.research.google.com/github/iscsteam/ai-chatbot_G-nayan/blob/main/rag_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To learn more about accelerating pandas on Colab, see the [10 minute guide](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_colab_demo.ipynb) or
 [US stock market data analysis demo](https://colab.research.google.com/github/rapidsai-community/showcase/blob/main/getting_started_tutorials/cudf_pandas_stocks_demo.ipynb).

In [5]:
! pip install langchain -q
! pip install langchain_community -q
! pip install langchain_groq -q
! pip install grandalf -q
! pip install numpy -q
! pip install pandas -q
! pip install sentence-transformers -q
! pip install faiss-cpu -q
! pip install groq -q
! pip install PyPDF2 -q
! pip install rank_bm25 langchain pinecone-client -q
! pip install pymupdf -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 79.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.5 MB/s eta 0:00:0

In [ ]:
!pip install --upgrade langchain langchain_chroma langchain_community \
    langchain_groq grandalf numpy pandas sentence-transformers \
    faiss-cpu langchain-deepseek-official groq PyPDF2 -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 18.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import langchain
import os
from groq import Groq
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
import glob
from rank_bm25 import BM25Okapi
import re
import getpass
from dotenv import load_dotenv
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from PyPDF2 import PdfReader
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from langchain.schema import Document
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory

In [ ]:

from langchain.agents import initialize_agent, AgentType

In [ ]:
import sys
print(f"Python Version: {sys.version}")
# Or, for a cleaner version string:
!python --version
import groq
print(groq.__version__)

Python Version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
Python 3.11.11
0.19.0


In [ ]:
import langchain
import groq
import rank_bm25
import PyPDF2
import dotenv
import langchain_community
import langchain_groq
import sys
print(f"Python Version: {sys.version}")
print("LangChain version:", langchain.__version__)
print("Groq version:", groq.__version__)

print("PyPDF2 version:", PyPDF2.__version__)

print("LangChain Community version:", langchain_community.__version__)
print("LangChain Groq version:", langchain_groq.__version__)

Python Version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
LangChain version: 0.3.20
Groq version: 0.19.0
PyPDF2 version: 3.0.1
LangChain Community version: 0.3.19
LangChain Groq version: 0.2.5


In [3]:
import os
from dotenv import load_dotenv
env_file_path = '/content/drive/MyDrive/env_varaible/.env'

# # Load the .env file
# load_dotenv(env_file_path)
# groq_api_key=API_KEY = os.environ["groq_api_key"]
# groq_api_key


ModuleNotFoundError: No module named 'dotenv'

In [ ]:
pdf_path=r"/content/drive/MyDrive/Resumes"
pdf_files = glob.glob(os.path.join(pdf_path, "*.pdf"))
for pdf_file in pdf_files:
  def extract_text_from_pdf(pdf_file):
        reader = PdfReader(pdf_file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
        return text
  text = extract_text_from_pdf(pdf_path)
  def clean_whitespace(text):
      """Removes extra whitespace, tabs, and newlines."""
      text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces, tabs, newlines with a single space
      text = text.strip()  # Remove leading/trailing whitespace
      return text

  cleaned_text = clean_whitespace(text)

  def handle_bullet_points(text):
      """Replaces Unicode bullet points with standard bullets."""
      text = text.replace('\uf0b7', '• ')  # Replace with a standard bullet and a space
      return text
  cleaned_text = handle_bullet_points(cleaned_text)
  # Initialize the Groq client
  client = Groq(api_key=groq_api_key)  # Use os.getenv to fetch the environment variable
  # Define the function description
  resume_function_description = {
      "name": "extract_resume_details",
      "description": "Extract metadata and sections from a resume",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {"type": "string", "description": "Full name of the candidate"},
              "email": {"type": "string", "description": "Email address"},
              "phone_number": {"type": "string", "description": "Phone number"},
              "summary": {"type": "string", "description": "Summary section of the resume complete summary field dont miss anything"},
              "skills": {"type": "string", "description": "Skills section of the resume technical skills"},
              "projects": {"type": "string", "description": "Projects section full session description bring everything"},
              "experience": {"type": "string", "description": "Work experience section of the resume"},
              "education": {"type": "string", "description": "Education section of the resume"},
          },
          "required": ["name", "email", "phone_number"],
      },
  }

  # Define the PromptTemplate
  system_prompt_template = PromptTemplate(
      input_variables=["resume_text"],  # Placeholder for dynamic input
      template="""
  You are an AI Assistant. You will be given a resume text and need to convert it into a structured JSON format as described by the function `extract_resume_details`.
  If the data for a field is not provided in the resume, return "not provided" for that field.
  Strictly adhere to the format and properties defined in `extract_resume_details`
  for the "summary" field extract the complete summary section,
  For the "projects" field, extract **all details** including:
  - The project title.
  - A **detailed** description of the project.
  - Tools and technologies used.
  - Methodologies implemented.
  - Outcomes or results achieved.
  - dont forget any information and extract all the details
  Do not omit any details provided in the resume about the projects Ensure all available information is captured completely.**

  Resume Text:
  {resume_text}
      """
  )


  # Generate the formatted system prompt
  system_prompt = system_prompt_template.format(resume_text=cleaned_text)

  # Call the model with the formatted system prompt
  chat_completion = client.chat.completions.create(
      model="llama-3.3-70b-versatile",
      messages=[
          {
              "role": "system",
              "content": system_prompt
          },
          {
              "role": "user",
              "content": cleaned_text
          }
      ],
      functions=[resume_function_description],  # Pass the function description here
      function_call={"name": "extract_resume_details"},  # Tell the model to use this function
      max_tokens=2000,
      temperature=0.4,
  )
  # Parse the JSON response from the model
  try:
      function_args = json.loads(chat_completion.choices[0].message.function_call.arguments)
      #print(json.dumps(function_args, indent=4))  # Print nicely formatted JSON
  except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
      print("Raw response content:", chat_completion.choices[0].message.content)  # Print raw content if JSON parsing fails
  except AttributeError as e:
      print(f"Function call arguments not found in response: {e}")
      print("Raw response content:", chat_completion.choices[0].message.content)  #

  meta_data = {
      "name": function_args.get("name"),
      "email": function_args.get("email"),
      "phone_number": function_args.get("phone_number"),
  }
  doc=function_args
  content = f"""
  name: {function_args.get('name')}
  phone: {function_args.get('phone')}
  email: {function_args.get('email')}
  summary: {function_args.get('summary')}
  education: {function_args.get('education')}
  experience: {function_args.get('experience')}
  skills: {function_args.get('skills')}
  projects: {function_args.get('projects')}
  """
  docs=Document(page_content=content,metadata=meta_data)
documents=[doc]
# Initialize the HuggingFace Embedding model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectore_store_FAISS=FAISS.from_documents([documents],embedding=embed_model)
vectore_store_FAISS.save_local("faiss_index")
# Save FAISS index and metadata to Google Drive
save_path = "/content/drive/MyDrive/fasis_db/resume_index_faiss"
vectore_store_FAISS.save_local(save_path)
print(f"FAISS index saved to {save_path}")

In [ ]:
docs=Document(page_content=content,metadata=meta_data)
documents=[doc]
# Initialize the HuggingFace Embedding model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectore_store_FAISS=FAISS.from_documents([documents],embedding=embed_model)
vectore_store_FAISS.save_local("faiss_index")
# Save FAISS index and metadata to Google Drive
save_path = "/content/drive/MyDrive/fasis_db/resume_index_faiss"
vectore_store_FAISS.save_local(save_path)
print(f"FAISS index saved to {save_path}")
# Initialize the HuggingFace Embedding model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# # Embed the content (as a list of documents)
# vectors = embed_model.embed_documents([content])

# # Print the resulting vector
# print("Generated Vector:")
# print(vectors[0])  # S
#embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectore_store_FAISS=FAISS.from_documents([documents],embedding=embed_model)
vectore_store_FAISS.save_local("faiss_index")
# Save FAISS index and metadata to Google Drive
save_path = "/content/drive/MyDrive/fasis_db/resume_index_faiss"
vectore_store_FAISS.save_local(save_path)

print(f"FAISS index saved to {save_path}")

In [ ]:
docs=Document(page_content=content,metadata=meta_data)
documents=[doc]
# Initialize the HuggingFace Embedding model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
vectore_store_FAISS=FAISS.from_documents([documents],embedding=embed_model)
vectore_store_FAISS.save_local("faiss_index")
# Save FAISS index and metadata to Google Drive
save_path = "/content/drive/MyDrive/fasis_db/resume_index_faiss"
vectore_store_FAISS.save_local(save_path)
print(f"FAISS index saved to {save_path}")
# Step 1: Extract text from the resume
from PyPDF2 import PdfReader
import re
pdf_path=r"/content/drive/MyDrive/Resumes/Salvapathi Naidu_Gen-AI.pdf"


def extract_text_from_pdf(pdf_file):
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text
text = extract_text_from_pdf(pdf_path)
def clean_whitespace(text):
    """Removes extra whitespace, tabs, and newlines."""
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces, tabs, newlines with a single space
    text = text.strip()  # Remove leading/trailing whitespace
    return text

cleaned_text = clean_whitespace(text)

def handle_bullet_points(text):
    """Replaces Unicode bullet points with standard bullets."""
    text = text.replace('\uf0b7', '• ')  # Replace with a standard bullet and a space
    return text

cleaned_text = handle_bullet_points(cleaned_text)



# Initialize the Groq client
client = Groq(api_key=groq_api_key)  # Use os.getenv to fetch the environment variable

# Define the function description
resume_function_description = {
    "name": "extract_resume_details",
    "description": "Extract metadata and sections from a resume",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "Full name of the candidate"},
            "email": {"type": "string", "description": "Email address"},
            "phone_number": {"type": "string", "description": "Phone number"},
            "summary": {"type": "string", "description": "Summary section of the resume complete summary field dont miss anything"},
            "skills": {"type": "string", "description": "Skills section of the resume technical skills"},
            "projects": {"type": "string", "description": "Projects section full session description bring everything"},
            "experience": {"type": "string", "description": "Work experience section of the resume"},
            "education": {"type": "string", "description": "Education section of the resume"},
        },
        "required": ["name", "email", "phone_number"],
    },
}

# Define the PromptTemplate
system_prompt_template = PromptTemplate(
    input_variables=["resume_text"],  # Placeholder for dynamic input
    template="""
You are an AI Assistant. You will be given a resume text and need to convert it into a structured JSON format as described by the function `extract_resume_details`.
If the data for a field is not provided in the resume, return "not provided" for that field.
Strictly adhere to the format and properties defined in `extract_resume_details`
for the "summary" field extract the complete summary section,
For the "projects" field, extract **all details** including:
- The project title.
- A **detailed** description of the project.
- Tools and technologies used.
- Methodologies implemented.
- Outcomes or results achieved.
- dont forget any information and extract all the details
Do not omit any details provided in the resume about the projects Ensure all available information is captured completely.**

Resume Text:
{resume_text}
    """
)


# Generate the formatted system prompt
system_prompt = system_prompt_template.format(resume_text=cleaned_text)

# Call the model with the formatted system prompt
chat_completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": cleaned_text
        }
    ],
    functions=[resume_function_description],  # Pass the function description here
    function_call={"name": "extract_resume_details"},  # Tell the model to use this function
    max_tokens=2000,
    temperature=0.5,
)
# Parse the JSON response from the model
try:
    function_args = json.loads(chat_completion.choices[0].message.function_call.arguments)
    #print(json.dumps(function_args, indent=4))  # Print nicely formatted JSON
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
    print("Raw response content:", chat_completion.choices[0].message.content)  # Print raw content if JSON parsing fails
except AttributeError as e:
    print(f"Function call arguments not found in response: {e}")
    print("Raw response content:", chat_completion.choices[0].message.content)  #

meta_data = {
    "name": function_args.get("name"),
    "email": function_args.get("email"),
    "phone_number": function_args.get("phone_number"),
}
doc=function_args
content = f"""
name: {function_args.get('name')}
phone: {function_args.get('phone')}
email: {function_args.get('email')}
summary: {function_args.get('summary')}
education: {function_args.get('education')}
experience: {function_args.get('experience')}
skills: {function_args.get('skills')}
projects: {function_args.get('projects')}

"""
docs1=Document(page_content=content,metadata=meta_data)
docs1

In [ ]:

# Step 1: Extract text from the resume
from PyPDF2 import PdfReader
import re
pdf_path=r"/content/drive/MyDrive/Resumes"
pdf_files = glob.glob(os.path.join(pdf_path, "*.pdf"))
for pdf_file in pdf_files:
  def extract_text_from_pdf(pdf_file):
      reader = PdfReader(pdf_file)
      text = ""
      for page in reader.pages:
          text += page.extract_text()
      return text
  text = extract_text_from_pdf(pdf_file)
  def clean_whitespace(text):
      """Removes extra whitespace, tabs, and newlines."""
      text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces, tabs, newlines with a single space
      text = text.strip()  # Remove leading/trailing whitespace
      return text

  cleaned_text_1 = clean_whitespace(text)

  def handle_bullet_points(text):
      """Replaces Unicode bullet points with standard bullets."""
      text = text.replace('\uf0b7', '• ')  # Replace with a standard bullet and a space
      return text

  cleaned_text = handle_bullet_points(cleaned_text_1)



  # Initialize the Groq client
  client = Groq(api_key=groq_api_key)  # Use os.getenv to fetch the environment variable

  # Define the function description
  resume_function_description = {
      "name": "extract_resume_details",
      "description": "Extract metadata and sections from a resume",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {"type": "string", "description": "Full name of the candidate"},
              "email": {"type": "string", "description": "Email address"},
              "phone_number": {"type": "string", "description": "Phone number"},
              "summary": {"type": "string", "description": "Summary section of the resume complete summary field dont miss anything"},
              "skills": {"type": "string", "description": "Skills section of the resume technical skills"},
              "projects": {"type": "string", "description": "Projects section full session description bring everything"},
              "experience": {"type": "string", "description": "Work experience section of the resume"},
              "education": {"type": "string", "description": "Education section of the resume"},
          },
          "required": ["name", "email", "phone_number"],
      },
  }

  # Define the PromptTemplate
  system_prompt_template = PromptTemplate(
      input_variables=["resume_text"],  # Placeholder for dynamic input
      template="""
  You are an AI Assistant. You will be given a resume text and need to convert it into a structured JSON format as described by the function `extract_resume_details`.
  If the data for a field is not provided in the resume, return "not provided" for that field.
  Strictly adhere to the format and properties defined in `extract_resume_details`
  for the "summary" field extract the complete summary section,
  For the "projects" field, extract **all details** including:
  - The project title.
  - A **detailed** description of the project.
  - Tools and technologies used.
  - Methodologies implemented.
  - Outcomes or results achieved.
  - dont forget any information and extract all the details
  Do not omit any details provided in the resume about the projects Ensure all available information is captured completely.**

  Resume Text:
  {resume_text}
      """
  )


  # Generate the formatted system prompt
  system_prompt = system_prompt_template.format(resume_text=cleaned_text)

  # Call the model with the formatted system prompt
  chat_completion = client.chat.completions.create(
      model="llama-3.3-70b-versatile",
      messages=[
          {
              "role": "system",
              "content": system_prompt
          },
          {
              "role": "user",
              "content": cleaned_text
          }
      ],
      functions=[resume_function_description],  # Pass the function description here
      function_call={"name": "extract_resume_details"},  # Tell the model to use this function
      max_tokens=20000,
      temperature=0.5,
  )
  # Parse the JSON response from the model
  try:
      function_args = json.loads(chat_completion.choices[0].message.function_call.arguments)
      #print(json.dumps(function_args, indent=4))  # Print nicely formatted JSON
  except json.JSONDecodeError as e:
      print(f"Error decoding JSON: {e}")
      print("Raw response content:", chat_completion.choices[0].message.content)  # Print raw content if JSON parsing fails
  except AttributeError as e:
      print(f"Function call arguments not found in response: {e}")
      print("Raw response content:", chat_completion.choices[0].message.content)  #

  meta_data = {
      "name": function_args.get("name"),
      "email": function_args.get("email"),
      "phone_number": function_args.get("phone_number"),
  }
  doc=function_args
  content = f"""
  name: {function_args.get('name')}
  phone: {function_args.get('phone')}
  email: {function_args.get('email')}
  summary: {function_args.get('summary')}
  skills: {function_args.get('skills')}
  education: {function_args.get('education')}
  experience: {function_args.get('experience')}
  projects: {function_args.get('projects')}


  """
  doc=Document(page_content=content,metadata=meta_data)
  PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
  PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
  PINECONE_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]
  original_doc=doc
  original_doc=[original_doc]
  pinecone = PineconeClient(api_key=PINECONE_API_KEY,
                          environment=PINECONE_ENVIRONMENT)
  index_name = PINECONE_INDEX_NAME

  embeddings = PineconeEmbeddings(model="multilingual-e5-large")
  docs_to_vectorstore = PineconeVectorStore.from_documents(
          documents=original_doc,
          index_name="resume-retrival",
          embedding=embeddings
      )

NameError: name 'PineconeEmbeddings' is not defined

In [ ]:
query = "Front-end & back-end development, Product Engineering, Agile software development, Test-driven development, Team Management & Leadership, Azure, AWS, Docker, Kubernetes, Jenkins, MongoDB, PostgreSQL, RabbitMQ, Python, Django, Flask, FastAPI, AI, Data Science, Spark, Kafka, Git, gRPC"
# Perform the vector search
vector_search_results = docs_to_vectorstore.similarity_search(query,k=10)

for idx, doc in enumerate(vector_search_results):
    print(f"Result {idx + 1}:")
    print(f"Metadata: {doc.metadata}")  # Extract metadata
    print(f"Content: {doc.page_content[:1000000]}")  # Display first 500 characters of the content
    print("-" * 90)  # Separator for better readability

In [ ]:
def hybrid_search(query, top_k=10):
    query_embedding = get_embedding(query)

    # Dense Search (Pinecone)
    pinecone_index = pinecone.Index("your-index")
    dense_results = pinecone_index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

    # Sparse Search (BM25)
    bm25_scores = bm25.get_scores(query.split())
    sparse_results = sorted(enumerate(bm25_scores), key=lambda x: x[1], reverse=True)[:top_k]

    # Fusion & Re-ranking
    final_results = rerank(dense_results, sparse_results)

    return final_results


In [ ]:
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
embed_model

In [ ]:

content = f"""
name: {resumr_jason_formate.get('name')}
phone: {resumr_jason_formate.get('phone')}
email: {resumr_jason_formate.get('email')}
summary: {resumr_jason_formate.get('summary')}
education: {resumr_jason_formate.get('education')}
experience: {resumr_jason_formate.get('experience')}
projects: {resumr_jason_formate.get('projects')}


"""
doc=Document(page_content=content,metadata=meta_data)
doc

In [ ]:
docs=list([doc])
docs

In [ ]:
# Load the existing FAISS index
load_path = "/content/drive/MyDrive/fasis_db/faiss_index"
vectore_store_FAISS = FAISS.load_local(load_path, embeddings=embed_model, allow_dangerous_deserialization=True)

# New document to append
new_doc = doc #Document(page_content="This is new content.", metadata={"source": "new_source"})

# Add the new document to the FAISS index
vectore_store_FAISS.add_documents([new_doc])

# Save the updated index
vectore_store_FAISS.save_local(load_path)

print("New data appended and FAISS index updated!")

New data appended and FAISS index updated!


In [ ]:
# Path to the saved FAISS index
load_path = "/content/drive/MyDrive/fasis_db/faiss_index"
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
# Load the FAISS vector store
vectore_store_FAISS_db = FAISS.load_local(load_path, embeddings=embed_model,allow_dangerous_deserialization=True)

print("FAISS index loaded successfully!")
# Example to query the vector store
query = "experienced in jango and flask and  Kubernetes Jenkins"
results = vectore_store_FAISS_db.similarity_search(query,k=2)

for result in results:
    print(f"Metadata: {result.metadata}")
    print(f"Document Content: {result.page_content}")
    #print(f"Similarity Score: {result.score}")  # Print the similarity score

    print("----")


In [ ]:
import os
from langchain_pinecone import PineconeEmbeddings
from langchain_community.vectorstores import Pinecone
os.environ["PINECONE_API_KEY"] = "pcsk_bx41y_64e6FxaTR6A1StMSez4LYUKZJ69HzYyEUW6CvU9TuqCuW3UFRuWqJGhyhg67AaD"
model = PineconeEmbeddings(model="multilingual-e5-large",show_progress_bar=True)
model

PineconeEmbeddings(model='multilingual-e5-large', batch_size=96, query_params={'input_type': 'query', 'truncation': 'END'}, document_params={'input_type': 'passage', 'truncation': 'END'}, dimension=1024, show_progress_bar=True, pinecone_api_key=SecretStr('**********'))

In [ ]:
import os
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import CohereEmbeddings
from langchain_community.vectorstores import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from pinecone import Pinecone as PineconeClient
# import requests
# from pinecone.core.client import ServerlessSpec

In [ ]:
PINECONE_API_KEY=os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT=os.environ["PINECONE_ENVIRONMENT"]
PINECONE_INDEX_NAME=os.environ["PINECONE_INDEX_NAME"]

In [ ]:
! pip install pinecone-client -q


In [ ]:
import pinecone
from langchain.vectorstores import Pinecone
from langchain.embeddings import PineconeEmbeddings
#from langchain.embeddings.pinecone import PineconeEmbeddings
from langchain.retrievers import PineconeHybridSearchRetriever
from rank_bm25 import BM25Okapi

In [ ]:
original_doc1=docs1
original_doc1=[original_doc1]
original_doc1

In [ ]:

# docs=[doc]

pinecone = PineconeClient(api_key=PINECONE_API_KEY,
                          environment=PINECONE_ENVIRONMENT)
index_name = PINECONE_INDEX_NAME
embeddings = PineconeEmbeddings(model="multilingual-e5-large")
docs_to_vectorstore = PineconeVectorStore.from_documents(
          documents=original_doc,
          index_name="resume-retrival",
          embedding=embeddings
      )
#
# Define BM25 Sparse Encoder (using the rank_bm25 package)
tokenized_corpus = [doc.page_content.lower().split() for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)

class BM25SparseEncoder:
    def __init__(self, bm25):
        self.bm25 = bm25

    def encode(self, query):
        """Compute BM25 scores for a given query"""
        tokenized_query = query.lower().split()
        scores = self.bm25.get_scores(tokenized_query)
        return {i: score for i, score in enumerate(scores) if score > 0}

# Instantiate the BM25 Sparse Encoder
sparse_encoder = BM25SparseEncoder(bm25)

# # Set up Hybrid Search Retriever
retriever = PineconeHybridSearchRetriever(
    vectorstore=docs_to_vectorstore,
    sparse_encoder=sparse_encoder,
    alpha=0.5
)
# Perform a query
query = "micro services and flask and python"
results = retriever.get_relevant_documents(query, top_k=3)

# Display results
for idx, doc in enumerate(results):
    print(f"Result {idx + 1}:")
    print(f"Metadata: {doc.metadata}")  # Extract metadata
    print(f"Content: {doc.page_content[:1000]}")  # Display first 1000 characters of content
    print("-" * 80)  # Separator for better readability


In [ ]:
from rank_bm25 import BM25Okapi
# Tokenize documents
tokenized_corpus = [doc.lower().split() for doc in original_doc]

# Initialize BM25 model
bm25 = BM25Okapi(tokenized_corpus)

# Define Sparse Encoder (BM25)
class BM25SparseEncoder:
    def __init__(self, bm25):
        self.bm25 = bm25

    def encode(self, query):
        """Compute BM25 scores for a given query"""
        tokenized_query = query.lower().split()
        scores = self.bm25.get_scores(tokenized_query)
        return {i: score for i, score in enumerate(scores) if score > 0}

# Instantiate Sparse Encoder
sparse_encoder = BM25SparseEncoder(bm25)


In [ ]:
# Assuming `original_doc` is a list of documents you want to insert into Pinecone
docs_to_vectorstore = PineconeVectorStore.from_documents(
    documents=original_doc,
    index_name="resume-retrival",
    embedding=embeddings
)

# Initialize Hybrid Search Retriever
retriever = PineconeHybridSearchRetriever(
    embeddings=embeddings,
    vectorstore=docs_to_vectorstore,
    sparse_encoder=sparse_encoder,  # Use BM25 or TF-IDF encoder here
    alpha=0.5,  # Balance between dense and sparse search
)

In [ ]:
query = "micro services and flask and python"
results = retriever.get_relevant_documents(query, top_k=2)

for idx, doc in enumerate(results):
    print(f"Result {idx + 1}:")
    print(f"Metadata: {doc.metadata}")  # Extract metadata
    print(f"Content: {doc.page_content[:1000]}")  # Display first 1000 characters
    print("-" * 80)  # Separator for better readability


In [ ]:

query = "micro services and flask and python"
# Perform the vector search
vector_search_results = docs_to_vectorstore.similarity_search(query,k=2)

for idx, doc in enumerate(vector_search_results):
    print(f"Result {idx + 1}:")
    print(f"Metadata: {doc.metadata}")  # Extract metadata
    print(f"Content: {doc.page_content[:1000]}")  # Display first 500 characters of the content
    print("-" * 80)  # Separator for better readability

In [ ]:
# @ CHat Bot gen-AI
## CHat Bot gen-A

In [ ]:
! pip install --upgrade langchain langchain-community groq -q
! pip install langchain-groq -q
! pip install pymupdf -q
import os

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 59.6 MB/s eta 0:00:00


In [ ]:
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community import embeddings
from langchain.schema import Document
from langchain.vectorstores import FAIS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import getpass
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.schema import SystemMessage, HumanMessage
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage
import json
from langchain.memory import ConversationBufferWindowMemory
from langchain.document_loaders import PyMuPDFLoader  # PDF Loader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:

# ✅ Step 1: Load PDF and Split Pages as Single Chunks
pdf_path = "/content/drive/MyDrive/g-nayan_vector_db/G-Nayan- DR AI Soln - Project Review Doc_Ethics Committe_Gandhi Medical College (1).pdf"  # Replace with your actual PDF path
loader = PyMuPDFLoader(pdf_path)  # Load PDF
documents = loader.load()  # Extract all pages as separate documents

# ✅ Step 2: Assign Metadata (Page Number)
for i, doc in enumerate(documents):
    doc.metadata["page_number"] = i + 1  # Track page numbers

# ✅ Step 3: Initialize HuggingFace Embeddings
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# ✅ Step 4: Store in FAISS Vector Database
vector_store_FAISS = FAISS.from_documents(documents, embedding=embed_model)

# ✅ Step 5: Save FAISS Index
faiss_save_path = "/content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss"

vector_store_FAISS.save_local(faiss_save_path)
print(f"FAISS index saved to {faiss_save_path}")


FAISS index saved to /content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss


In [ ]:
import os
from langchain.document_loaders import PyMuPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# ✅ Step 1: Define Paths
faiss_save_path = "/content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss"
new_pdf_path = "/content/drive/MyDrive/g-nayan_vector_db/G-Nayan- DR AI Soln - Project Review Doc_Ethics Committe_Gandhi Medical College (1).pdf"  # Update with actual path

# ✅ Step 2: Initialize Embedding Model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# ✅ Step 3: Load Existing FAISS Index (If Exists)
if os.path.exists(faiss_save_path):
    try:
        vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model, allow_dangerous_deserialization=True)
        print("✅ Loaded existing FAISS index.")
    except Exception as e:
        print(f"⚠️ Error loading FAISS index: {e}")
        vector_store_FAISS = None
else:
    print("❌ No existing FAISS index found. A new one will be created.")
    vector_store_FAISS = None

# ✅ Step 4: Load New PDF and Process It
new_loader = PyMuPDFLoader(new_pdf_path)  # Load new PDF
new_documents = new_loader.load()  # Extract pages

# Assign metadata (Page Number)
for i, doc in enumerate(new_documents):
    doc.metadata["page_number"] = i + 1  # Adjust as needed

# ✅ Step 5: Convert New Documents to FAISS Vectors
new_vector_store = FAISS.from_documents(new_documents, embedding=embed_model)

# ✅ Step 6: Merge with Existing FAISS Index (if available)
if vector_store_FAISS:
    vector_store_FAISS.merge_from(new_vector_store)
else:
    vector_store_FAISS = new_vector_store  # If no existing index, use the new one

# ✅ Step 7: Save Updated FAISS Index
vector_store_FAISS.save_local(faiss_save_path)
print(f"✅ FAISS index updated and saved to {faiss_save_path}")


✅ Loaded existing FAISS index.
✅ FAISS index updated and saved to /content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss


In [ ]:
# ✅ Load the FAISS Index
vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model,allow_dangerous_deserialization=True) # ✅ Enable safe deserialization)

# ✅ Query Example
query = "g-nayana"
query_embedding = embed_model.embed_query(query)

# ✅ Search FAISS for Similar Chunks
retrieved_docs = vector_store_FAISS.similarity_search(query, k=3)


# ✅ Print Retrieved Documents
for doc in retrieved_docs:
    print(f"Page{doc.page_content}\n")


In [ ]:
! pip install rank-bm25 -q

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
from rank_bm25 import BM25Okapi
from langchain.vectorstores import FAISS
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
faiss_save_path = "/content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss"
# ✅ Load FAISS Vector Store
vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model, allow_dangerous_deserialization=True)

# ✅ Load Text Data for BM25 (Assuming 'documents' is a list of text chunks)
documents = [doc.page_content for doc in vector_store_FAISS.docstore._dict.values()]
tokenized_docs = [doc.split() for doc in documents]  # Tokenize for BM25
bm25 = BM25Okapi(tokenized_docs)

# ✅ Hybrid Search Function
def hybrid_search(query, k=3, lambda_weight=0.5):
    query_embedding = embed_model.embed_query(query)

    # ✅ FAISS Dense Search
    dense_results = vector_store_FAISS.similarity_search(query, k=k)

    # ✅ BM25 Sparse Search
    bm25_scores = bm25.get_scores(query.split())
    top_k_bm25 = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:k]
    sparse_results = [documents[i] for i in top_k_bm25]

    # ✅ Combine Results (Weighted Ranking)
    hybrid_results = []
    for i in range(k):
        dense_score = (1 - lambda_weight) * (k - i)  # Higher rank = higher weight
        sparse_score = lambda_weight * (k - i)  # Higher rank = higher weight
        hybrid_results.append((dense_results[i].page_content, dense_score))
        hybrid_results.append((sparse_results[i], sparse_score))

    # ✅ Sort & Return Final Ranked Results
    hybrid_results.sort(key=lambda x: x[1], reverse=True)
    return [res[0] for res in hybrid_results[:k]]

# ✅ Query Example
query = user_input#"what is G-nayan or g-nayana"
results = hybrid_search(query, k=5)
# results
# # # ✅ Print Retrieved Documents
# # for idx, res in enumerate(results):
# #     print(f"{idx+1}. {res}\n")
# context_text = "\n\n".join(results)
# context_text
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
# Initialize Memory with Window Size 3
context_text = "\n\n".join(results)
if not api_key:
    api_key = getpass.getpass("Enter API key for Groq: ")
    os.environ["GROQ_API_KEY"] = api_key  # Store it in the environment

# Initialize the Groq chat model
chat_model = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=200)
system_prompt_template = PromptTemplate.from_template(
    "Summarize the following documents relevant to the query{context} less than 200 words in a good formate "
)
# Create a system message from the template
system_message = SystemMessage(content=system_prompt_template.format(context=context_text))

print("your Assistant is ready! Type 'exit' or 'quit' to end the chat.")

# Chat loop
while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ["exit", "quit","thank you","tq"]:
        print("G-Nayan: Goodbye!")
        break
    # Get response from the chatbot with stored memory
    response = chat_model.invoke([
        system_message,
        HumanMessage(content=user_input)
    ])

    # Store user input and bot response in memory
    print(f"G-Nayan: {response.content}")

In [ ]:
# Load API key from .env file
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
# Initialize Memory with Window Size 3
context_text = "\n\n".join(results)
if not api_key:
    api_key = getpass.getpass("Enter API key for Groq: ")
    os.environ["GROQ_API_KEY"] = api_key  # Store it in the environment

# Initialize the Groq chat model
chat_model = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=200)
system_prompt_template = PromptTemplate.from_template(
    "Summarize the following documents relevant to the query{context} less than 200 words in a good formate "
)
# Create a system message from the template
system_message = SystemMessage(content=system_prompt_template.format(context=context_text))

print("your Assistant is ready! Type 'exit' or 'quit' to end the chat.")

# Chat loop
while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ["exit", "quit","thank you","tq"]:
        print("G-Nayan: Goodbye!")
        break
    # Get response from the chatbot with stored memory
    response = chat_model.invoke([
        system_message,
        HumanMessage(content=user_input)
    ])

    # Store user input and bot response in memory
    print(f"G-Nayan: {response.content}")

In [ ]:



# ✅ Initialize Embedding Model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# ✅ Load FAISS Vector Store
faiss_save_path = "/content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss"
vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model, allow_dangerous_deserialization=True)

# ✅ Load Text Data for BM25 (Assuming 'documents' is a list of text chunks)
documents = [doc.page_content for doc in vector_store_FAISS.docstore._dict.values()]
tokenized_docs = [doc.split() for doc in documents]  # Tokenize for BM25
bm25 = BM25Okapi(tokenized_docs)

# ✅ Hybrid Search Function
def hybrid_search(query, k=3, lambda_weight=0.5):
    query_embedding = embed_model.embed_query(query)

    # ✅ FAISS Dense Search
    dense_results = vector_store_FAISS.similarity_search(query, k=k)

    # ✅ BM25 Sparse Search
    bm25_scores = bm25.get_scores(query.split())
    top_k_bm25 = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:k]
    sparse_results = [documents[i] for i in top_k_bm25]

    # ✅ Combine Results (Weighted Ranking)
    hybrid_results = []
    for i in range(len(dense_results)):  # Avoid index errors
        dense_score = (1 - lambda_weight) * (k - i)  # Higher rank = higher weight
        hybrid_results.append((dense_results[i].page_content, dense_score))

    for i in range(len(sparse_results)):
        sparse_score = lambda_weight * (k - i)  # Higher rank = higher weight
        hybrid_results.append((sparse_results[i], sparse_score))

    # ✅ Sort & Return Final Ranked Results
    hybrid_results.sort(key=lambda x: x[1], reverse=True)
    return [res[0] for res in hybrid_results[:k]]

# ✅ Initialize Groq LLM Model
chat_model = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=200)

def rag_pipeline(user_input,results):
  user_input = input("\nYou: ").strip()
  # ✅ Perform Hybrid Search
  results = hybrid_search(user_input, k=5)
  context_text = "\n\n".join(results)

  # ✅ System Prompt for Summarization
  system_prompt_template = PromptTemplate.from_template(
  "Summarize the following documents relevant to the query:\n"
  "{context}\n\n"
  "Provide the summary in less than 200 words, ensuring:\n"
  "- No hallucinations.\n"
  "- No repetition from the referenced document.\n"
  "- Maintain factual accuracy"
  )
  system_message = SystemMessage(content=system_prompt_template.format(context=context_text))
  response = chat_model.invoke([
      system_message,
      HumanMessage(content=user_input)
  ])
  return f"\nG-Nayan: {response.content}"
while True:
    user_input = input("You: ")
    if user_input in ["exit", "quit", "thank you", "tq", "bye"]:
      print("G-Nayan: Goodbye! just ping me hi you wana kind of help")
      break  # Exit before calling chat_bot
    response =rag_pipeline(user_input,results)
    print(response)



In [ ]:
# # ✅ Load Text Data for BM25 (Assuming 'documents' is a list of text chunks)
# documents = [doc.page_content for doc in vector_store_FAISS.docstore._dict.values()]
# tokenized_docs = [doc.split() for doc in documents]  # Tokenize for BM25
# bm25 = BM25Okapi(tokenized_docs)

# # ✅ Hybrid Search Function
# def hybrid_search(query, k=5, lambda_weight=0.5):
#     query_embedding = embed_model.embed_query(query)

#     # ✅ FAISS Dense Search
#     dense_results = vector_store_FAISS.similarity_search(query, k=k)

#     # ✅ BM25 Sparse Search
#     bm25_scores = bm25.get_scores(query.split())
#     top_k_bm25 = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:k]
#     sparse_results = [documents[i] for i in top_k_bm25]

#     # ✅ Combine Results (Weighted Ranking)
#     hybrid_results = []
#     for i in range(len(dense_results)):  # Avoid index errors
#         dense_score = (1 - lambda_weight) * (k - i)  # Higher rank = higher weight
#         hybrid_results.append((dense_results[i].page_content, dense_score))

#     for i in range(len(sparse_results)):
#         sparse_score = lambda_weight * (k - i)  # Higher rank = higher weight
#         hybrid_results.append((sparse_results[i], sparse_score))

#     # ✅ Sort & Return Final Ranked Results
#     hybrid_results.sort(key=lambda x: x[1], reverse=True)
#     return [res[0] for res in hybrid_results[:k]]

In [ ]:
! pip install fastapi uvicorn nest-asyncio pyngrok -q


In [ ]:
# Load API key from .env file
!ngrok authtoken 2c7p2kLhZ8kcRfIsbSg1lzVWOqV_84ytnNJmQVngJaowK2CvZ

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    api_key = getpass.getpass("Enter API key for Groq: ")
    os.environ["GROQ_API_KEY"] = api_key  # Store it in the environment
# ✅ Initialize FastAPI
app = FastAPI()
# ✅ Initialize Embedding Model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# ✅ Load FAISS Vector Store
faiss_save_path = "/content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss"
vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model, allow_dangerous_deserialization=True)


def similarity_search(query,k):
  vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model,allow_dangerous_deserialization=True) # ✅ Enable safe deserialization)
  query_embedding = embed_model.embed_query(query)
    # ✅ Search FAISS for Similar Chunks
  retrieved_docs = vector_store_FAISS.similarity_search(query,k)
  return retrieved_docs


# ✅ Initialize Groq LLM Model
chat_model_rag = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=250)

def rag_pipeline(user_input):
  # ✅ Perform Hybrid Search
  results = similarity_search(user_input, k=5)
  context_text = "\n\n".join([doc.page_content for doc in results])

  system_prompt_template = PromptTemplate.from_template(
    "Summarize the following documents relevant to the query:\n"
    "{context}\n\n"
    "Provide the summary in less than 250 words, ensuring:\n"
    "- No hallucinations.\n"
    "- No repetition from the referenced document.\n"
    "- Maintain factual accuracy.\n"
    "- If there is no similarity between query and generation, just return 'non'."
)
  system_message = SystemMessage(content=system_prompt_template.format(context=context_text))
  response = chat_model_rag.invoke([
      system_message,
      HumanMessage(content=user_input)
  ])
  return f"\nG-Nayan: {response.content}"
report = {
    "patient_id": "ys jagan 420",
    "age":45,
    "phone _number":7832697672,
    "email_id":"gandhi_hospital@gmail.com",
    "left_eye": {
        #"predicted_class": 1,
        "stage": "Mild Diabetic Retinopathy",
        # "confidence": 96.52,
        "explanation": "Mild NPDR (Nonproliferative Diabetic Retinopathy):\n Microaneurysms have been detected, which are small, localized dilations of blood vessels in the retina this finding suggests a moderate chance of progression if not monitored and managed properly.'",
        "note": "Your eye is in the sslightly effected."
    },
    "right_eye": {
        #"predicted_class": 3,
        "stage": "Proliferative Retinopathy",
        # "confidence": 97.66,
        "explanation": "Proliferative Diabetic Retinopathy (PDR)Neovascularization has been observed, characterized by the growth of new, abnormal blood vessels on the retina Additionally, cotton wool spots may be present, indicating localized retinal ischemia These findings carry a significant risk of vision loss and complications, necessitating urgent medical intervention",
        "note": "Take care of your eye."
    }}
# ✅ Convert the report to a JSON string for display
report_json = json.dumps(report, indent=2)
def chat_bot(user_input):
  user_input = user_input.strip()
  # Initialize the Groq chat model
  chat_model = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=300)
  system_prompt_template = PromptTemplate.from_template(
    "You are an AI chatbot called G-Nayan. "
    "You specialize in explaining diabetic retinopathy in detail. "

    "If the user greets you, ask them if they want an analysis of their medical report. "
    "If they respond with 'yes' or 'ha', analyze the provided medical report and summarize "
    "both the left and right eye conditions. Provide better insights, ensuring they are not just a repetition of the report. "
    "Mention the patient's name (from patient_id) and the hospital in a structured format. "
    "This analysis should only be given once unless the user explicitly asks for it again.\n\n"

    "Additionally, based on the report analysis, suggest:\n"
    "- Suitable food recommendations.\n"
    "- Symptoms to monitor.\n"
    "- The type of doctor the patient should consult.\n\n"

    "Report Details:\n{report}\n\n"

    "Rules:\n"
    "- Never go out of your role.\n"
    "- Do not explicitly mention that you are an AI unless asked.\n"
    "- Do not repeatedly state 'As a specialist in diabetic retinopathy.'\n"
    "- If asked an inappropriate or unrelated question, respond with: "
    "'I am limited to medical diabetic retinopathy discussions only.'"
)

  # Create a system message from the template
  system_message = SystemMessage(content=system_prompt_template.format(report=report_json))
  # Get response from the chatbot with stored memory
  response = chat_model.invoke([
      system_message,
      HumanMessage(content=user_input)
  ])
  return f"\nG-Nayan: {response.content}"
# ✅ Define WebSocket Endpoint
@app.websocket("/chat")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        try:
            user_input = await websocket.receive_text()
            if user_input.lower() in ["exit", "quit", "thank you", "bye"]:
                await websocket.send_text("G-Nayan: Goodbye! Ping me 'hi' if you need help.")
                break

            # 🔹 Handle RAG Queries
            if any(keyword in user_input for keyword in ["documentation", "document", "doc"]):
                response = rag_pipeline(user_input)
                if not response or response.strip().lower() == "non":
                    response = chat_bot(user_input)
            else:
                response = chat_bot(user_input)

            await websocket.send_text(f"G-Nayan: {response}")

        except Exception as e:
            await websocket.send_text(f"G-Nayan: An error occurred: {str(e)}")
            break

    await websocket.close()
# Start ngrok tunnel
# ✅ Start ngrok tunnel
ngrok_tunnel = ngrok.connect(8000)
print(f"Public WebSocket URL: {ngrok_tunnel.public_url}/chat")

# ✅ Run FastAPI Server
if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


# while True:
#     user_input = input("You: ").strip().lower()

#     if user_input in ["exit", "quit", "thank you", "tq", "bye"]:
#         print("G-Nayan: Goodbye! Just ping me 'hi' if you need any help.")
#         break

#     # 🔹 Step 1: Check if the query is related to documents
#     if any(keyword in user_input for keyword in ["documentation", "document", "doc"]):
#         response = rag_pipeline(user_input)

#         # 🔹 Step 2: If RAG fails (returns empty or "non"), fallback to chatbot
#         if not response or response.strip().lower() == "non":
#             response = chat_bot(user_input)
#     else:
#         # 🔹 Default to chatbot for general queries
#         response = chat_bot(user_input)

#     print(response)



# # ✅ Define tools for Langchain
# chatbot_tool = Tool(
#     name="ChatBot",
#     func=lambda query: chat_bot(query),
#     description="Use for general conversational queries like greetings and normal conversations. hi or hello general questions"
# )
# rag_tool = Tool(
#     name="RAG Retrieval",
#     func=lambda query: rag_pipeline(query),
#     description="Use when a user asks for factual or document-based information retrieved from the RAG pipeline."
# )

In [ ]:
!ngrok authtoken 2c7p2kLhZ8kcRfIsbSg1lzVWOqV_84ytnNJmQVngJaowK2CvZ


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:


# ✅ Load environment variables
load_dotenv()
os.environ["GROQ_API_KEY"] = "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"
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    api_key = getpass.getpass("Enter API key for Groq: ")
    os.environ["GROQ_API_KEY"] = api_key  # Store it in the environment

# ✅ Initialize Embedding Model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# ✅ Load FAISS Vector Store
faiss_save_path = "/content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss"
vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model, allow_dangerous_deserialization=True)

def similarity_search(query, k):
    vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model, allow_dangerous_deserialization=True)
    query_embedding = embed_model.embed_query(query)
    retrieved_docs = vector_store_FAISS.similarity_search(query, k)
    return retrieved_docs

# ✅ Initialize Groq LLM Model
chat_model_rag = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=250)

def rag_pipeline(user_input):
    results = similarity_search(user_input, k=5)
    context_text = "\n\n".join([doc.page_content for doc in results])

    system_prompt_template = PromptTemplate.from_template(
        "Summarize the following documents relevant to the query:\n"
        "{context}\n\n"
        "Provide the summary in less than 250 words, ensuring:\n"
        "- No hallucinations.\n"
        "- No repetition from the referenced document.\n"
        "- Maintain factual accuracy.\n"
        "- If there is no similarity between query and generation, just return 'non'."
    )
    system_message = SystemMessage(content=system_prompt_template.format(context=context_text))
    response = chat_model_rag.invoke([
        system_message,
        HumanMessage(content=user_input)
    ])
    return f"\nG-Nayan: {response.content}"

# ✅ Define patient report
report = {
    "patient_id": "ys jagan 420",
    "age": 45,
    "phone_number": 7832697672,
    "email_id": "gandhi_hospital@gmail.com",
    "left_eye": {
        "stage": "Mild Diabetic Retinopathy",
        "explanation": "Mild NPDR (Nonproliferative Diabetic Retinopathy):\n Microaneurysms have been detected, which are small, localized dilations of blood vessels in the retina. This finding suggests a moderate chance of progression if not monitored and managed properly.",
        "note": "Your eye is slightly affected."
    },
    "right_eye": {
        "stage": "Proliferative Retinopathy",
        "explanation": "Proliferative Diabetic Retinopathy (PDR): Neovascularization has been observed, characterized by the growth of new, abnormal blood vessels on the retina. Additionally, cotton wool spots may be present, indicating localized retinal ischemia. These findings carry a significant risk of vision loss and complications, necessitating urgent medical intervention.",
        "note": "Take care of your eye."
    }
}

# ✅ Convert the report to a JSON string for display
report_json = json.dumps(report, indent=2)

def chat_bot(user_input):
    user_input = user_input.strip()
    chat_model = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=300)

    system_prompt_template = PromptTemplate.from_template(
        "You are an AI chatbot called G-Nayan. "
        "You specialize in explaining diabetic retinopathy in detail. "
        "If the user greets you, ask them if they want an analysis of their medical report. "
        "If they respond with 'yes' or 'ha', analyze the provided medical report and summarize "
        "both the left and right eye conditions. Provide better insights, ensuring they are not just a repetition of the report. "
        "Mention the patient's name (from patient_id) and the hospital in a structured format. "
        "This analysis should only be given once unless the user explicitly asks for it again.\n\n"
        "Additionally, based on the report analysis, suggest:\n"
        "- Suitable food recommendations.\n"
        "- Symptoms to monitor.\n"
        "- The type of doctor the patient should consult.\n\n"
        "Report Details:\n{report}\n\n"
        "Rules:\n"
        "- Never go out of your role.\n"
        "- Do not explicitly mention that you are an AI unless asked.\n"
        "- Do not repeatedly state 'As a specialist in diabetic retinopathy.'\n"
        "- If asked an inappropriate or unrelated question, respond with: "
        "'I am limited to medical diabetic retinopathy discussions only.'"
    )
    system_message = SystemMessage(content=system_prompt_template.format(report=report_json))
    response = chat_model.invoke([
        system_message,
        HumanMessage(content=user_input)
    ])
    return f"\nG-Nayan: {response.content}"
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory

# ✅ Define tools for Langchain
chatbot_tool = Tool(
    name="ChatBot",
    func=chat_bot,
      description=(
        "Use this tool for general conversational queries like greetings, normal discussions, "
        "or when the user asks for an analysis of their diabetic retinopathy report. "
        "This tool is responsible for summarizing patient reports and providing relevant insights."
    ))

rag_tool = Tool(
    name="RAG Retrieval",
    func=rag_pipeline,
     description=(
        "Use this tool when the user asks for document-based information from the RAG pipeline. "
        "It is best suited for answering queries about 'G-Nayan', 'Diabetic Retinopathy', "
        "or when searching for food recommendations and doctors related to diabetic retinopathy."
    ))
llm = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=300)
memory = ConversationBufferMemory(memory_key="chat_history")

# ✅ Create an agent
agent = initialize_agent(
    llm=llm,
    tools=[chatbot_tool,rag_tool],  # Add both tools
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Allows the agent to decide based on descriptions
    verbose=False,
    max_execution_time=25,
    memory=memory
)

# ✅ Chat loop using the agent
while True:
    user_input = input("You: ").strip().lower()

    if user_input in ["exit", "quit", "thank you", "tq", "bye"]:
        print("G-Nayan: Goodbye! Just ping me 'hi' if you need any help.")
        break

    # 🔹 Let the agent handle tool selection
    response = agent.run(user_input)

    print(f"\nG-Nayan: {response}")



You: hii

G-Nayan: Agent stopped due to iteration limit or time limit.


KeyboardInterrupt: Interrupted by user

In [ ]:
re

In [ ]:
for_query = ChatGroq(model_name="llama3-8b-8192", temperature=0.0, max_tokens=100)

def llm_query_rewrite(query):
    """Rewrite user queries using LLM for better retrieval performance."""
    prompt = f"Rewrite the following search query in a well-structured, concise format:\n\n'{query}'"

    try:
        response = for_query.invoke(prompt)
        return response.content # Ensure clean output
    except Exception as e:
        print(f"LLM Query Rewrite Error: {str(e)}")  # Log errors
        return query  # Fall back to original query if LLM fails
        print(response.content)
llm_query_rewrite("what is diabeeetic retinopthy")

'Here is a rewritten version of the search query in a well-structured and concise format:\n\n"Definition of Diabetic Retinopathy"\n\nor\n\n"What is Diabetic Retinopathy?"\n\nor\n\n"Information on Diabetic Retinopathy"\n\nThis rewritten query is more specific and clear, making it easier for search engines to understand what you\'re looking for.'

In [ ]:
for_query = ChatGroq(model_name="llama3-8b-8192", temperature=0.0, max_tokens=50)
def llm_query_rewrite(query):
    """Rewrite user queries using LLM for better retrieval performance."""
    prompt = f"""
    Rewrite the following search query in a well-structured, concise format.
    Directly return **only** 2-3 query variations, with each query on a new line.
    No explanations, no numbering, no extra words.

    Input: {query}
    Output:
    """

    try:
        response = for_query.invoke(prompt)

        # ✅ Ensure we extract text correctly
        if hasattr(response, "content") and isinstance(response.content, str):
            queries = response.content.strip().split("\n")
            return [q.strip() for q in queries if q.strip()]  # Clean spaces and blank lines

        return [query]  # Fallback to original query
    except Exception as e:
        print(f"LLM Query Rewrite Error: {str(e)}")
        return [query]  # Fallback to original query

# 🔍 Test case
# cleaned_queries = llm_query_rewrite("what are foods to be taken for overcoming daibatic retinopathy")
# cleaned_queries

def similarity_search(query, k):
    # Load FAISS index
    vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model,allow_dangerous_deserialization=True)

    # Rewrite the query for better retrieval
    cleaned_queries = llm_query_rewrite(query)
    print(cleaned_queries)
    # Convert the first rewritten query into an embedding
    query_embedding = embed_model.embed_query(cleaned_queries[0])

    # Perform similarity search using the query vector
    retrieved_docs = vector_store_FAISS.similarity_search_by_vector(query_embedding, k)

    return retrieved_docs

# Test the function
print(similarity_search("what is Diabetic renouhd", k=3))




['what is diabetic retinopathy', 'what is diabetic retinopaty', 'diabetic retinopathy definition']
[Document(id='eae816d1-1d28-4a2d-b415-f73b91ca61ca', metadata={'producer': 'Adobe PDF Library 11.0', 'creator': 'Adobe InDesign CC 2014 (Macintosh)', 'creationdate': '2016-08-04T12:03:10-05:00', 'source': '/content/drive/MyDrive/g-nayan_vector_db/fact_sheet_22_diabetic_retinopathy_new.pdf', 'file_path': '/content/drive/MyDrive/g-nayan_vector_db/fact_sheet_22_diabetic_retinopathy_new.pdf', 'total_pages': 4, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2016-08-04T12:03:12-05:00', 'trapped': '', 'modDate': "D:20160804120312-05'00'", 'creationDate': "D:20160804120310-05'00'", 'page': 0, 'page_number': 1}, page_content='Diabetic Retinopathy: Diabetic retinopathy \n(pronounced ret in OP uh thee) is a complication of diabetes \nthat causes damage to the blood vessels of the retina— \nthe light-sensitive tissue that lines the back part of the eye, \na

In [ ]:
for_query = ChatGroq(model_name="llama3-8b-8192", temperature=0.0, max_tokens=50)
def llm_query_rewrite(query):

    prompt = f"""
    Rewrite the following search query in a well-structured and correct grammer and expand the query, concise format.

    No explanations, no numbering, no extra words.

    Input: {query}
    Output:
    """

    try:
        response = for_query.invoke(prompt)

        # ✅ Ensure we extract text correctly
        if hasattr(response, "content") and isinstance(response.content, str):
            queries = response.content.strip().split("\n")
            return [q.strip() for q in queries if q.strip()]  # Clean spaces and blank lines

        return [query]  # Fallback to original query
    except Exception as e:
        print(f"LLM Query Rewrite Error: {str(e)}")
        return [query]  # Fallback to original query

# 🔍 Test case
cleaned_queries = llm_query_rewrite("what is g-nayan and what does it do")
cleaned_queries


['G-Nayan and its functions.']

In [ ]:
# while True:
#     user_input = input("You: ").strip().lower()

#     if user_input in ["exit", "quit", "thank you", "tq", "bye"]:
#         print("G-Nayan: Goodbye! Just ping me 'hi' if you need any help.")
#         break

#     if any(keyword in user_input for keyword in ["documentation", "document", "doc"]):
#         response = rag_pipeline(user_input)
#         if not response or response.strip().lower() == "non":
#             response = chat_bot(user_input)
#     else:
#         response = chat_bot(user_input)

#     print(response)

# # ✅ Define tools for Langchain
# chatbot_tool = Tool(
#     name="ChatBot",
#     func=lambda query: chat_bot(query),
#     description="Use for general conversational queries like greetings and normal conversations. hi or hello general questions"
# )

# rag_tool = Tool(
#     name="RAG Retrieval",
#     func=lambda query: rag_pipeline(query),
#     description="Use when a user asks for factual or document-based information retrieved from the RAG pipeline."
# )

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# ✅ Load environment variables
import os
import getpass
import json
from dotenv import load_dotenv
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.schema import SystemMessage, HumanMessage
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory


load_dotenv()
os.environ["GROQ_API_KEY"] = "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" #"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"
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    api_key = getpass.getpass("Enter API key for Groq: ")
    os.environ["GROQ_API_KEY"] = api_key  # Store it in the environment

# ✅ Initialize Embedding Model
embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# ✅ Load FAISS Vector Store
faiss_save_path = "/content/drive/MyDrive/g-nayan_vector_db/dr_index_faiss"
vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model, allow_dangerous_deserialization=True)
for_query = ChatGroq(model_name="llama3-8b-8192", temperature=0.0, max_tokens=100)
for_query = ChatGroq(model_name="llama3-8b-8192", temperature=0.0, max_tokens=50)
def llm_query_rewrite(query):
    """Rewrite user queries using LLM for better retrieval performance."""
    prompt = f"""
    Rewrite the following search query in a well-structured, concise format.
    Directly return **only** 2-3 query variations, with each query on a new line.
    No explanations, no numbering, no extra words.

    Input: {query}
    Output:
    """

    try:
        response = for_query.invoke(prompt)

        # ✅ Ensure we extract text correctly
        if hasattr(response, "content") and isinstance(response.content, str):
            queries = response.content.strip().split("\n")
            return [q.strip() for q in queries if q.strip()]  # Clean spaces and blank lines

        return [query]  # Fallback to original query
    except Exception as e:
        print(f"LLM Query Rewrite Error: {str(e)}")
        return [query]  # Fallback to original query



for_query = ChatGroq(model_name="llama3-8b-8192", temperature=0.0, max_tokens=50)
def llm_query_rewrite(query):
    """Rewrite user queries using LLM for better retrieval performance."""
    prompt = f"""
    Rewrite the following search query in a well-structured, concise format.
    Directly return **only** 2-3 query variations, with each query on a new line.
    No explanations, no numbering, no extra words.

    Input: {query}
    Output:
    """

    try:
        response = for_query.invoke(prompt)

        # ✅ Ensure we extract text correctly
        if hasattr(response, "content") and isinstance(response.content, str):
            queries = response.content.strip().split("\n")
            return [q.strip() for q in queries if q.strip()]  # Clean spaces and blank lines

        return [query]  # Fallback to original query
    except Exception as e:
        print(f"LLM Query Rewrite Error: {str(e)}")
        return [query]  # Fallback to original query

# 🔍 Test case
# cleaned_queries = llm_query_rewrite("what are foods to be taken for overcoming daibatic retinopathy")
# cleaned_queries

def similarity_search(query, k):
    # Load FAISS index
    vector_store_FAISS = FAISS.load_local(faiss_save_path, embed_model,allow_dangerous_deserialization=True)

    # Rewrite the query for better retrieval
    cleaned_queries = llm_query_rewrite(query)
    # print(cleaned_queries)
    # Convert the first rewritten query into an embedding
    query_embedding = embed_model.embed_query(cleaned_queries[0])

    # Perform similarity search using the query vector
    retrieved_docs = vector_store_FAISS.similarity_search_by_vector(query_embedding, k)

    return retrieved_docs



# ✅ Initialize Groq LLM Model
chat_model_rag = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=250)

def rag_pipeline(user_input):
    results = similarity_search(user_input, k=3)
    context_text = "\n\n".join([doc.page_content for doc in results])

    system_prompt_template = PromptTemplate.from_template(
        "Summarize the following documents relevant to the query and give full description of the query without truncate:\n"
        "{context}\n\n"
        "Provide the summary in less than 400 words, ensuring:\n"
        "- No hallucinations.\n"
        "- No repetition from the referenced document.\n"
        "- Maintain factual accuracy.\n"
        "- If there is no similarity between query and generation, just return 'non'."
    )
    system_message = SystemMessage(content=system_prompt_template.format(context=context_text))
    response = chat_model_rag.invoke([
        system_message,
        HumanMessage(content=user_input)
    ])
    return response.content

# ✅ Define patient report
report = {
    "patient_id": "ys jagan 420",
    "age": 45,
    "phone_number": 7832697672,
    "email_id": "gandhi_hospital@gmail.com",
    "left_eye": {
        "stage": "Mild Diabetic Retinopathy",
        "explanation": "Mild NPDR (Nonproliferative Diabetic Retinopathy):\n Microaneurysms have been detected, which are small, localized dilations of blood vessels in the retina. This finding suggests a moderate chance of progression if not monitored and managed properly.",
        "note": "Your eye is slightly affected."
    },
    "right_eye": {
        "stage": "Proliferative Retinopathy",
        "explanation": "Proliferative Diabetic Retinopathy (PDR): Neovascularization has been observed, characterized by the growth of new, abnormal blood vessels on the retina. Additionally, cotton wool spots may be present, indicating localized retinal ischemia. These findings carry a significant risk of vision loss and complications, necessitating urgent medical intervention.",
        "note": "Take care of your eye."
    }
}

# ✅ Convert the report to a JSON string for display
report_json = json.dumps(report, indent=2)
# "If the user greets you, ask them if they want an analysis of their medical {report}. "
#         "If they respond with 'yes' or 'ha', analyze the provided medical report and summarize \ "
#         "both the left and right eye conditions. Provide better insights, ensuring they are not just a repetition of the report. "
#         "Mention the patient's name (from patient_id) and the hospital in a structured format. "
#         "This analysis should only be given once unless the user explicitly asks for it again.\n\n"
#         "Report Details:\n{report}\n\n"
#         "Rules:\n"
def chat_bot(user_input):
    user_input = user_input.strip()
    chat_model = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=250)
    system_prompt = """
    You are G-Nayan, an AI chatbot specializing in general conversations, RAG-integrated chatbot functionalities, and discussions related to diabetic retinopathy.

## **Guidelines for Responses:**
1. **Scope of Interaction:**
   - You engage in general conversation greetings.
   - You explain your role as an AI chatbot with a RAG pipeline integration.
   - You provide accurate and concise responses related to diabetic retinopathy.

2. **Medical Queries:**
   - Provide reliable and relevant information about diabetic retinopathy.
   - Avoid offering medical diagnoses or personalized treatment recommendations.
   - If a user asks unrelated or inappropriate medical questions, respond with:
     *"I can only assist with general discussions and diabetic retinopathy-related topics."*

3. **Behavioral Constraints:**
   - Never explicitly state that you are an AI unless asked.
   -do not go out of your role.
   - Do not repeatedly mention: *"As a specialist in diabetic retinopathy."*
   - If asked about unrelated or inappropriate topics, respond with: any other topics except diabetic retinopathy
     *"I am limited to discussions on diabetic retinopathy and general chatbot interactions."*
   - Always prioritize clear, factual, and concise responses.

4. **Conversation Flow:**
   - Maintain professionalism while keeping the interaction engaging.
   - Avoid redundant or unnecessary explanations.
   - Ensure responses are informative yet easy to understand.

Follow these rules strictly to ensure consistency and user satisfaction.

    """

    response = chat_model.invoke([
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_input)
    ])
    return response.content

from langchain.tools import Tool

# ✅ Define tools for Langchain
chatbot_tool = Tool(
    name="ChatBot",
    func=chat_bot,
   description=(
        "Use this ability for general conversational queries like greetings and normal discussions. "
        "If the user greets you, respond accordingly using the chat_bot function. "
        "This ability is for handling casual conversation and answering non-medical questions. "
        "For a diabetic retinopathy report analysis, redirect to the system prompt for reports. "
        "Do not use this ability for medical queries; those should be handled by RAG. "
    )
)

rag_tool = Tool(
    name="RAG Retrieval",
    func=rag_pipeline,
    description=(
    "Use this tool when the user seeks document-based information from the RAG pipeline. "
    "It is ideal for answering queries related to 'G-Nayan', 'Diabetic Retinopathy', 'Retinopathy', "
    "its causes, symptoms, stages, diagnosis, and treatment options. "
    "Additionally, use this tool when the user inquires about food recommendations, lifestyle changes, "
    "or relevant medical specialists for diabetic retinopathy management."
))


Agent_llm = ChatGroq(model_name="llama3-8b-8192", temperature=0.3, max_tokens=300)
memory = ConversationBufferMemory(memory_key="chat_history")

# ✅ Create an agent with updated configuration
agent = initialize_agent(
    llm=Agent_llm,
    tools=[chatbot_tool,rag_tool],
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=False,
    max_iterations=2,  # Limit the number of thinking iterations
    early_stopping_method="generate",
    return_intermediate_steps=False,
    return_only_outputs=True,  #,  # Ensure it doesn't get stuck
    handle_parsing_errors=True,  # Handle errors gracefully
    memory=memory
)

while True:
    user_input = input("You: ").strip().lower()

    # ✅ Handle greetings dynamically and prevent unnecessary agent calls
    if user_input in ["hi", "hello", "hey", "hai"]:
        print("G-Nayan: Hello! Hi am your AI bot ...How can I assist you?")
        continue  # ✅ Prevents the agent from processing a simple greeting

    if user_input in ["exit", "quit", "thank you", "bye","tq"]:
        print("G-Nayan: Goodbye! Just ping me 'hi' if you need any help and you know diabetic retinopathy follow the link: https://www.iscstech.com")
        break

    try:
        response = agent.run(user_input)  # ✅ Now returns only chatbot responses
        print(f"\nG-Nayan: {response}")
    except Exception as e:
        print(f"\nG-Nayan: I'm sorry, I encountered an issue. How else can I help you?")


<ipython-input-9-d4d7609c832a>:223: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-9-d4d7609c832a>:226: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


You: haiii


<ipython-input-9-d4d7609c832a>:252: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(user_input)  # ✅ Now returns only chatbot responses



G-Nayan: Hello! I'm G-Nayan, a chatbot designed to assist with general conversations and provide information on diabetic retinopathy. I'm integrated with a RAG pipeline, which allows me to respond to a wide range of topics while ensuring accuracy and reliability. How can I help you today?
You: how are you..

G-Nayan: It's nice to have a friendly conversation! I'm happy to chat with you about general topics or answer any questions you may have about diabetic retinopathy.
You: what is G-nayan explain me

G-Nayan: Here is the final answer:

Question: what is g-nayan explain me

Thought: The user is asking about G-Nayan, which is a term related to diabetic retinopathy. I should use the RAG Retrieval tool to provide information on G-Nayan.

Action: RAG Retrieval
Action Input: "What is G-Nayan?"
You: hat is G-Nayan?

G-Nayan: G-NAYAN is an artificial intelligence (AI) system designed to detect and diagnose diabetic retinopathy (DR) from retinal images.
You: G-Nayan deatiled explanation plea